# ETL de ventas de SUMUP

Extraeremos las ventas a través de la API de SUMUP y añadiremos esa información a las bases de datos de ventas Nikarit en Bigquery y en Firebase. <br>
¿Qué es SUMUP? - Sumup es la aplicación que usamos para registrar ventas cuando estamos en mercadillos. Registramos tanto ventas manuales como ventas a través del datáfono de SUMUP <br>
<br>
La idea es integrar estas ventas con las ventas online en una misma base de datos

In [1]:
#to open cred files
import os
#cleaning and operations
import pandas as pd
import numpy as np
#to get values of API
import json
import unidecode
import requests

In [2]:
from time import sleep

In [3]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [4]:
#bigquery
import pandas_gbq
from google.cloud import bigquery

In [5]:
#FIREBASE
from firebase_admin import credentials,storage,firestore,initialize_app

In [6]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

True

In [84]:
## OPEN CONECTION TO BIGQUERRY
gbq_client = bigquery.Client()
dataset_id = "{}.contoan".format(client.project)
dataset = bigquery.Dataset(dataset_id)

In [8]:
filename=os.environ['FIREBASE_FILENAME']
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase = initialize_app(cred)

db = firestore.client()

## NOS CONECTAMOS A SUMUP

In [9]:
#cogemos la ruta del archivo con las credenciales de sumup
sumup_file = os.environ['SUMUP']

In [10]:
#abrimos el archivo y lo cargamos en un diccionario
creds_sumup = json.load(open(sumup_file,'r'))

In [11]:
#endpoints de autentificación
authorize_url = creds_sumup["authorize_url"]
token_url = creds_sumup["token_url"]

In [12]:
client_id = creds_sumup["client_id"]
client_secret = creds_sumup["client_secret"]

In [13]:
callback_uri = creds_sumup["redirect_uris"]

In [14]:
scopes = ['transactions.history','user.profile','user.profile_readonly']

info sobre las transacciones - # https://developer.sumup.com/rest-api/#tag/Transactions

In [15]:
#para conseguir todas las transacciones
trans_hist = "https://api.sumup.com/v0.1/me/transactions/history"

#para conseguir más información de cada una de las transacciones
transaction_url = "https://api.sumup.com/v0.1/me/transactions"

### AUTENTIFICACION EN SUMUP PARA OBTENER CÓDIGO
Tal y como describe en documentación, la api de sumup, para poder recibir un token de acceso necesita <br>
que te autentiques como usuario para poder darte un token de acceso que puede permitirte <br>
acceder a la información de tu usuario

In [16]:
authorization_redirect_url = (authorize_url +
                              '?response_type=code&client_id=' +
                              client_id +
                              '&redirect_uri=' + 
                              callback_uri[0] +
                              '&scope='+scopes[0])

In [18]:
print(authorization_redirect_url)

In [20]:
#Añadimos el código que nos ha generado
code = input('enter code:\n')

In [21]:
#cojemos el token de acceso
data = {'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': callback_uri}
access_token_response = requests.post(token_url,
                                      data=data,
                                      verify=False,
                                      allow_redirects=False,
                                      auth=(client_id, client_secret))


In [22]:
tokens = json.loads(access_token_response.text)
access_token = tokens['access_token']

---- ya estamos autenticados

## COGEMOS LISTADO DE TRANSACCIONES

In [23]:
api_call_headers = {'Authorization': 'Bearer ' + access_token}
params = {
    "limit":1000
}
api_call_response = requests.get(trans_hist, 
                                 headers=api_call_headers, 
                                 params=params,
                                 verify=False)

In [25]:
'todo correcto' if api_call_response.status_code == 200 else 'La liaste'

'todo correcto'

In [26]:
ventas_sumup = api_call_response.json()

In [27]:
ventas_sumup = ventas_sumup['items']

In [28]:
df_ventas_sumup = pd.DataFrame(ventas_sumup)

In [29]:
df_ventas_sumup.shape

(647, 19)

### COGEMOS LA INFORMACION DE LOS PRODUCTOS QUE SE VENDIERON EN CADA VENTA

In [30]:
def get_info_trans(x):
    products = []
    lat = 0
    lon = 0
    transaction=''
    transaction_id=x
    params = {
        "id":transaction_id
    }
    api_call_response = requests.get(transaction_url, 
                                     headers=api_call_headers, 
                                     params=params,
                                     verify=False)
    status = api_call_response.status_code
    if status != 200:
        raise('error in {} - {}'.format(x,status))
    
    else:
        transaction = api_call_response.json()
        products = transaction['products']
        lat = transaction['lat']
        lon = transaction['lon']
    sleep(0.01)
    return pd.Series([products,lat,lon])

In [31]:
df_ventas_sumup[['products','lat','lon']] = df_ventas_sumup['transaction_id'].apply(lambda x: get_info_trans(x))

### TRANSFORMAMOS LA INFORMACION PARA QUE ESTE IGUAL QUE EN LA BBDD

#### COGEMOS listado de productos únicos 

In [32]:
l_all_prods = df_ventas_sumup['products'].to_list()

In [34]:
l_prods = []
for prods in l_all_prods:
    l_prods = l_prods+prods

In [35]:
df_prods = pd.DataFrame(l_prods)

In [36]:
df_prods[['name','price','vat_rate','price_with_vat']].drop_duplicates(subset=['name'])

,name,price,vat_rate,price_with_vat
0,,1.00,NaN,NaN
1,Nikarit,27.00,NaN,NaN
111,02 - Manteca de karité pura - 100 ml,6.61,0.21,8.0
112,04 - Crema de Manos,4.96,0.21,6.0
113,"01 - Pack ""La más hidratada del lugar""",14.05,0.21,17.0
115,Donacion (1€),1.00,0.00,1.0
117,03 - Bálsamo Labial,3.31,0.21,4.0
145,Donacion (10€),10.00,0.00,10.0


In [37]:
prods_name = df_prods['name'].unique()

### los productos que estan en blanco, es porque no se dieron de alta en la venta

In [38]:
prods_name

array(['', 'Nikarit', '02 - Manteca de karité pura - 100 ml',
       '04 - Crema de Manos', '01 - Pack "La más hidratada del lugar"',
       'Donacion (1€)', '03 - Bálsamo Labial', 'Donacion (10€)'],
      dtype=object)

In [39]:
d_prods_trans = {
    '':'',
    'Nikarit':'',
    '02 - Manteca de karité pura - 100 ml':'manteca_nb',
    '04 - Crema de Manos':'crema_nb', 
    '03 - Bálsamo Labial':'balsamo_nb',
    '01 - Pack "La más hidratada del lugar"':'packHidra_nb',
    'Donacion (1€)':'donation', 
    'Donacion (10€)':'donation'
}

In [40]:
def count_prods(x):
    mantecas = 0
    balsamos = 0
    cremas = 0
    donation= 0
    packs_hidra = 0
    amount = x['amount']
    prods = x['products']
    count_prods = {
        'manteca_nb':0,
        'crema_nb':0,
        'balsamo_nb':0,
        'packHidra_nb':0,
        'donation':0
    }
    
    #indicador para saber si se ha informado el producto en la venta
    match_prod = 0
    for pro in prods:
        ks = pro.keys()
        pp = d_prods_trans[pro['name']]
        tot_price = pro['total_with_vat'] if 'total_with_vat' in ks else pro['total_price']
        quant = pro['quantity']
        
        #En donaciones meteremos los euros de donacion
        if pp == 'donation' :
            count_prods[pp]=count_prods[pp]+tot_price
            match_prod = 1
        #el resto de productos meteremos la cantidad que se ha vendido
        elif pp != '':
            count_prods[pp]=count_prods[pp]+quant
            match_prod = 1
            
    #Para aquellos donde no se declaró producto intentaremos inducirlo por la cantidad vendida     
    if match_prod == 0:
        if amount % 17 == 0:
            count_prods['packHidra_nb'] = int(amount/17)
        elif amount % 8 == 0:
            count_prods['manteca_nb'] = int(amount/8)
        elif amount % 6 == 0:
            count_prods['crema_nb'] = int(amount/6)
        elif amount % 4 == 0:
            count_prods['balsamo_nb'] = int(amount/4)

    return pd.Series([count_prods[i] for i in count_prods])

In [41]:
df_ventas_sumup[['manteca_nb',
                'crema_nb',
                'balsamo_nb',
                 'packHidra_nb',
                'donation']]=df_ventas_sumup.apply(lambda x: count_prods(x),axis=1)

In [42]:
df_ventas_sumup[['packHidra_nb',
                 'manteca_nb',
                'crema_nb',
                'balsamo_nb',
                'donation']].sum()

packHidra_nb     64.0
manteca_nb      304.0
crema_nb        259.0
balsamo_nb      330.0
donation        323.0
dtype: float64

In [43]:
df_ventas_sumup.timestamp.max()

'2020-03-11T13:00:35.047Z'

In [44]:
df_ventas_sumup.timestamp.min()

'2019-05-18T06:33:08.623Z'

In [45]:
df_ventas_sumup['creation_date'] = df_ventas_sumup.timestamp
df_ventas_sumup['close_date'] = df_ventas_sumup.timestamp

In [46]:
trad_paytype = {'POS':'card','CASH':'cash'}

In [47]:
df_ventas_sumup['payment_type'] = df_ventas_sumup['payment_type'].apply(lambda x: trad_paytype[x])

In [48]:
df_ventas_sumup['manteca_unit_price'] = 8
df_ventas_sumup['crema_unit_price'] = 6
df_ventas_sumup['balsamo_unit_price'] = 4
df_ventas_sumup['packHidra_unit_price']= 17

In [49]:
refs = ['manteca','crema','balsamo','packHidra']

In [50]:
for pro in refs:
    df_ventas_sumup[pro+'_total_price'] = df_ventas_sumup[pro+'_unit_price'] * df_ventas_sumup[pro+'_nb']  

In [51]:
df_ventas_sumup['total_tax'] = df_ventas_sumup.apply(lambda x: sum([x[r+'_total_price']*0.21 for r in refs]),axis=1)

In [52]:
df_ventas_sumup['subtotal_price'] = df_ventas_sumup['amount']-df_ventas_sumup['donation']

In [53]:
df_ventas_sumup['origin'] = 'sumup'

### SUBIR A FIREBASE

In [54]:
sales_col = db.collection('Nikarit_Sales')

#### renombrar columnas para que coincidan con documentos anteriores de la bbdd

In [55]:
cols = df_ventas_sumup.columns

In [63]:
d_cols = dict(zip(cols,cols))

In [64]:
d_cols['user']='sumup_user'
d_cols['lat']='latitude'
d_cols['lon']='longitude'
d_cols['payment_type'] = 'gateway'
d_cols['amount'] = 'total_price'
d_cols['transaction_id'] = 'sumup_transaction_id'

In [65]:
df_ventas_sumup = df_ventas_sumup.rename(columns=d_cols)

In [66]:
fire_cols = ['sumup_user', 'sumup_transaction_id', 'creation_date', 'close_date', 'latitude', 'longitude', 'total_tax',
 'total_price', 'subtotal_price', 'gateway', 'origin', 'manteca_unit_price', 'crema_unit_price',
 'balsamo_unit_price', 'packHidra_unit_price','manteca_nb', 'crema_nb', 'balsamo_nb', 'packHidra_nb',
 'manteca_total_price', 'crema_total_price','balsamo_total_price', 'packHidra_total_price']

In [102]:
resta = set(fire_cols)-set(df_ventas_sumup.columns)
'estan todos los campos que queremos subir' if len(resta)==0 else 'faltan -'+str(resta)

'estan todos los campos que queremos subir'

### LOOP POR TODAS LAS VENTAS

In [103]:
sumup_ventas = df_ventas_sumup[fire_cols].T.to_dict()

In [104]:
for i in sumup_ventas:
    row = sumup_ventas[i]
    doc_acc = sales_col.add(row)[1]
    doc_acc_id = doc_acc.id
    doc_acc.update({'id':doc_acc_id})

## ACTUALIZAR BIGQUERY

In [83]:
df_ventas_sumup.to_gbq('{}.sumup_sales'.format(dataset.dataset_id),
                       project_id=dataset.project,
                       if_exists='replace')

1it [00:04,  4.53s/it]


In [88]:
dataset.project

'oan-nikarit'

In [90]:
ss = gbq_client.get_table('oan-nikarit.contoan.sumup_sales')

In [94]:
ss.num_rows

647